In [1]:
import pandas as pd

C:\Users\A200162055\Miniconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [10]:
df = pd.read_csv('../data/raw/spotify_2023.csv', encoding='latin-1')
print(f'The dataframe consists of {len(df)} rows and {len(df.columns)} columns.')

The dataframe consists of 953 rows and 24 columns.


# Data Cleaning

## Handling missing values and data types

**Business decisions:**
- Col streams: Replace invalid values with nan and convert to numeric
- Col in_shazam_charts: Value formatting, replace invalid values with nan and convert to numeric
- Col in_deezer_playlists: Value formatting, replace invalid values with nan and convert to numeric

In [3]:
col = 'streams'
df[col] = pd.to_numeric(df[col], errors='coerce')
df[col] = df[col].astype('Int64')

num_invalid = df[col].isna().sum()
if num_invalid > 0:
    print(f'Warning: {num_invalid} invalid entries found in column {col} and set to NaN.')

In [4]:
col = 'in_shazam_charts'
df[col] = df[col].str.replace(',', '', regex=False)
df[col] = pd.to_numeric(df[col], errors='coerce')
df[col] = df[col].astype('Int64')

num_invalid = df[col].isna().sum()
if num_invalid > 0:
    print(f'Warning: {num_invalid} invalid entries found in column {col} and set to NaN.')

In [5]:
col = 'in_deezer_playlists'
df[col] = df[col].str.replace(',', '', regex=False)
df[col] = pd.to_numeric(df[col], errors='coerce')
df[col] = df[col].astype('Int64')

## Handling duplicates

In [6]:
group_cols = ['track_name', 'artist(s)_name']

sum_cols = ['in_spotify_playlists', 'streams', 'in_apple_playlists', 'in_deezer_playlists', ]
avg_cols = ['artist_count', 'released_year', 'released_month', 'released_day', 'in_spotify_charts', 'in_apple_charts', 'in_deezer_charts', 'in_shazam_charts', 'bpm', 'danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'speechiness_%', 'liveness_%']
first_cols = ['key', 'mode']

In [7]:
agg_dict = {col: 'sum' for col in sum_cols}
agg_dict.update({col: 'mean' for col in avg_cols})
agg_dict.update({col: 'first' for col in first_cols})

df_aggregated = df.groupby(group_cols).agg(agg_dict).reset_index()

for col in avg_cols:
    df_aggregated[col] = df_aggregated[col].round().astype('Int64')


In [8]:
print(f'The dataframe consists of {len(df_aggregated)} rows and {len(df_aggregated.columns)} columns.')

The dataframe consists of 949 rows and 24 columns.


# Write cleaned data to processed folder

In [14]:
df_aggregated.to_csv('../data/processed/spotify_2023_cleaned.csv', index=False)